In [29]:
import pandas as pd
import numpy as np

In [30]:
risk = pd.read_csv("data/NRI_Table-Counties.csv")
risk.head()

,OID_,NRI_ID,STATE,STATEABBRV,STATEFIPS,COUNTY,COUNTYTYPE,COUNTYFIPS,STCOFIPS,POPULATION,...,WNTW_EALS,WNTW_EALR,WNTW_ALRB,WNTW_ALRP,WNTW_ALRA,WNTW_ALR_NPCTL,WNTW_RISKV,WNTW_RISKS,WNTW_RISKR,NRI_VER
0,68,C02013,Alaska,AK,2,Aleutians East,Borough,13,2013,3374,...,21.603219,Relatively Low,1.340420e-06,2.981170e-07,0.000000,73.939957,19576.208930,23.926185,Very Low,Mar-23
1,69,C02016,Alaska,AK,2,Aleutians West,Census Area,16,2016,5168,...,22.253172,Relatively Low,6.666330e-07,1.967190e-07,0.000000,58.712473,17666.429910,21.826281,Very Low,Mar-23
2,70,C02020,Alaska,AK,2,Anchorage,Municipality,20,2020,290985,...,44.599195,Relatively Low,1.639710e-09,1.107580e-08,0.000002,8.356546,44202.713670,42.729876,Relatively Low,Mar-23
3,71,C02050,Alaska,AK,2,Bethel,Census Area,50,2050,18633,...,66.914268,Relatively Moderate,5.230630e-07,4.266610e-07,0.000000,80.810894,160878.445800,75.119313,Relatively Moderate,Mar-23
4,72,C02060,Alaska,AK,2,Bristol Bay,Borough,60,2060,843,...,11.513463,Very Low,1.033850e-06,4.301380e-07,0.000000,74.589910,5566.346968,8.335985,Very Low,Mar-23


In [ ]:
## Florida & California overview
states_of_interest = ["FL", "CA"]


selected_df = risk[risk["STATEABBRV"].isin(states_of_interest)]



columns_of_interest = [
    "STATEABBRV",

    "COUNTY",
    "STCOFIPS",
    "HRCN_RISKV",
    "HRCN_RISKS",
    "HRCN_RISKR",
    "HRCN_EALT",
    "WFIR_RISKV",
    "WFIR_RISKS",
    "WFIR_RISKR",
    "WFIR_EALT",
]



df_2state = selected_df[columns_of_interest]



df_2state

,STATEABBRV,COUNTY,STCOFIPS,HRCN_RISKV,HRCN_RISKS,HRCN_RISKR,HRCN_EALT,WFIR_RISKV,WFIR_RISKS,WFIR_RISKR,WFIR_EALT
190,CA,Alameda,6001,NaN,NaN,Not Applicable,NaN,5.679284e+06,96.181992,Relatively Moderate,6.731674e+06
191,CA,Alpine,6003,NaN,NaN,Not Applicable,NaN,5.867949e+05,81.641744,Relatively Low,4.347271e+05
192,CA,Amador,6005,NaN,NaN,Not Applicable,NaN,8.339331e+06,97.391028,Relatively High,7.534156e+06
193,CA,Butte,6007,NaN,NaN,Not Applicable,NaN,1.704887e+07,98.790964,Relatively High,1.399520e+07
194,CA,Calaveras,6009,NaN,NaN,Not Applicable,NaN,2.358812e+07,99.427299,Relatively High,2.213133e+07
...,...,...,...,...,...,...,...,...,...,...,...
386,FL,Union,12125,3.497802e+06,78.420817,Relatively Low,2.744067e+06,7.752027e+05,84.123449,Relatively Low,6.043814e+05
387,FL,Volusia,12127,1.351775e+08,98.205473,Relatively High,1.130014e+08,1.253974e+07,98.218263,Relatively High,1.158749e+07
388,FL,Wakulla,12129,5.525302e+06,83.041723,Relatively Low,5.484523e+06,2.097036e+06,91.473115,Relatively Moderate,2.051197e+06
389,FL,Walton,12131,8.170312e+07,97.128757,Relatively High,7.979451e+07,5.696883e+05,81.482660,Relatively Low,4.808972e+05


**Top 3 risks each county is facing**

In [ ]:
## check the top 3 risks each county is facing
hazard_types = [

    "AVLN",
    "CFLD",
    "CWAV",
    "DRGT",
    "ERQK",
    "HAIL",
    "HWAV",
    "HRCN",
    "ISTM",
    "LNDS",
    "LTNG",

    "RFLD",
    "SWND",
    "TRND",
    "TSUN",
    "VLCN",
    "WFIR",
    "WNTW",
]



main_risks = pd.DataFrame()



for index, row in risk.iterrows():

    county_risks = {}

    for hazard in hazard_types:

        risk_score = row[f"{hazard}_RISKS"]

        if not pd.isna(risk_score):

            county_risks[hazard] = risk_score


    sorted_risks = sorted(county_risks.items(), key=lambda x: x[1], reverse=True)


    top_risks = sorted_risks[:3]


    main_risks.at[index, "STCOFIPS"] = row["STCOFIPS"]

    main_risks.at[index, "STATEABBRV"] = row["STATEABBRV"]

    main_risks.at[index, "COUNTY"] = row["COUNTY"]

    for i, (hazard, score) in enumerate(top_risks, 1):

        main_risks.at[index, f"Risk_{i}_Type"] = hazard

        main_risks.at[index, f"Risk_{i}_Score"] = score



main_risks

,STCOFIPS,STATEABBRV,COUNTY,Risk_1_Type,Risk_1_Score,Risk_2_Type,Risk_2_Score,Risk_3_Type,Risk_3_Score
0,2013.0,AK,Aleutians East,TSUN,72.972973,ERQK,69.933185,VLCN,61.797753
1,2016.0,AK,Aleutians West,TSUN,95.945946,ERQK,89.850461,VLCN,68.539326
2,2020.0,AK,Anchorage,ERQK,98.727331,AVLN,91.346154,WFIR,78.619154
3,2050.0,AK,Bethel,WFIR,78.396437,WNTW,75.119313,RFLD,74.673878
4,2060.0,AK,Bristol Bay,WFIR,40.216354,ERQK,37.957366,VLCN,32.584270
...,...,...,...,...,...,...,...,...,...
3226,56037.0,WY,Sweetwater,CWAV,73.560293,WFIR,62.360802,ERQK,57.842825
3227,56039.0,WY,Teton,AVLN,100.000000,LNDS,92.547382,ERQK,91.059497
3228,56041.0,WY,Uinta,WFIR,85.968820,ERQK,77.919185,WNTW,69.646834
3229,56043.0,WY,Washakie,WFIR,56.761056,CWAV,46.516067,ERQK,43.175310


In [33]:
## top 3 risks FL & CA are facing
main_risks[main_risks["STATEABBRV"].isin(["CA", "FL"])]

,STCOFIPS,STATEABBRV,COUNTY,Risk_1_Type,Risk_1_Score,Risk_2_Type,Risk_2_Score,Risk_3_Type,Risk_3_Score
190,6001.0,CA,Alameda,ERQK,99.936367,DRGT,98.759147,RFLD,98.122813
191,6003.0,CA,Alpine,LNDS,98.875683,WFIR,81.641744,AVLN,81.250000
192,6005.0,CA,Amador,DRGT,98.663697,WFIR,97.391028,HWAV,94.336621
193,6007.0,CA,Butte,DRGT,99.809100,WFIR,98.790964,ERQK,98.345530
194,6009.0,CA,Calaveras,WFIR,99.427299,LNDS,99.421780,DRGT,97.900095
...,...,...,...,...,...,...,...,...,...
386,12125.0,FL,Union,WFIR,84.123449,HRCN,78.420817,LTNG,60.038610
387,12127.0,FL,Volusia,LTNG,99.485199,WFIR,98.218263,HRCN,98.205473
388,12129.0,FL,Wakulla,WFIR,91.473115,HRCN,83.041723,RFLD,79.191855
389,12131.0,FL,Walton,HRCN,97.128757,LTNG,96.750322,RFLD,87.336939


In [34]:
df_clean_original = pd.read_csv("climate_price_merged_data.csv")

In [35]:
df_clean_original

,Unnamed: 0,state,county_risk,population,area,risk_value,risk_score,risk_ratng,county_match,county_price,...,black_pct,asian_pct,hispanic_pct,ed_pct,county_tax,tax_paid_2022,tax_trend,state_name,county_unemp,unemployment_rate
0,0,AK,Aleutians East,3374.0,15167.694230,8.949653e+05,4.231626,Very Low,aleutianseast,NaN,...,0.086842,0.225439,0.197076,15.935335,NaN,NaN,NaN,alaska,Aleutians East,1.4
1,1,AK,Aleutians West,5168.0,14258.993100,4.151810e+06,35.221126,Very Low,aleutianswest,NaN,...,0.049121,0.287080,0.127676,16.805628,NaN,NaN,NaN,alaska,Aleutians West,1.8
2,2,AK,Anchorage,290985.0,1966.338483,9.456571e+07,94.845689,Relatively Moderate,anchorage,Anchorage,...,0.047303,0.093670,0.090775,36.772161,NaN,NaN,NaN,alaska,Anchorage,3.3
3,3,AK,Bethel,18633.0,46015.507070,2.354071e+06,16.926503,Very Low,bethel,NaN,...,0.004500,0.010768,0.011090,12.728180,NaN,NaN,NaN,alaska,Bethel,9.4
4,4,AK,Bristol Bay,843.0,857.255664,1.055605e+05,0.159084,Very Low,bristolbay,NaN,...,0.007109,0.005924,0.053318,22.083981,NaN,NaN,NaN,alaska,Bristol Bay,8.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3262,3262,WY,Sweetwater,42238.0,10597.074000,1.189081e+06,6.617881,Very Low,sweetwater,Sweetwater,...,0.008303,0.006695,0.164577,19.391717,Sweetwater,1411.0,6.2,wyoming,Sweetwater,4.3
3263,3263,WY,Teton,23250.0,4259.833177,1.141156e+07,68.501432,Relatively Low,teton,Teton,...,0.002100,0.013244,0.141228,57.868509,Teton,4723.0,8.5,wyoming,Teton,2.3
3264,3264,WY,Uinta,20412.0,2109.220457,2.821381e+06,21.699014,Very Low,uinta,Uinta,...,0.002445,0.005281,0.097115,17.955112,Uinta,1317.0,10.6,wyoming,Uinta,3.8
3265,3265,WY,Washakie,7662.0,2265.341661,6.211963e+05,2.513522,Very Low,washakie,Washakie,...,0.001561,0.004294,0.139232,26.166329,Washakie,1317.0,0.9,wyoming,Washakie,5.0


In [36]:
main_risks

,STCOFIPS,STATEABBRV,COUNTY,Risk_1_Type,Risk_1_Score,Risk_2_Type,Risk_2_Score,Risk_3_Type,Risk_3_Score
0,2013.0,AK,Aleutians East,TSUN,72.972973,ERQK,69.933185,VLCN,61.797753
1,2016.0,AK,Aleutians West,TSUN,95.945946,ERQK,89.850461,VLCN,68.539326
2,2020.0,AK,Anchorage,ERQK,98.727331,AVLN,91.346154,WFIR,78.619154
3,2050.0,AK,Bethel,WFIR,78.396437,WNTW,75.119313,RFLD,74.673878
4,2060.0,AK,Bristol Bay,WFIR,40.216354,ERQK,37.957366,VLCN,32.584270
...,...,...,...,...,...,...,...,...,...
3226,56037.0,WY,Sweetwater,CWAV,73.560293,WFIR,62.360802,ERQK,57.842825
3227,56039.0,WY,Teton,AVLN,100.000000,LNDS,92.547382,ERQK,91.059497
3228,56041.0,WY,Uinta,WFIR,85.968820,ERQK,77.919185,WNTW,69.646834
3229,56043.0,WY,Washakie,WFIR,56.761056,CWAV,46.516067,ERQK,43.175310


In [37]:
# First merge based on the primary key 'STATEABBRV' and 'state'
merged_df = pd.merge(
    main_risks,
    df_clean_original,
    left_on="STATEABBRV",
    right_on="state",
    suffixes=("_main", "_clean"),
)

# Then filter based on the 2nd key 'COUNTY' and 'county_risk' for precise matching
final_df = merged_df[merged_df["COUNTY"] == merged_df["county_risk"]]

final_df

,STCOFIPS,STATEABBRV,COUNTY,Risk_1_Type,Risk_1_Score,Risk_2_Type,Risk_2_Score,Risk_3_Type,Risk_3_Score,Unnamed: 0,...,black_pct,asian_pct,hispanic_pct,ed_pct,county_tax,tax_paid_2022,tax_trend,state_name,county_unemp,unemployment_rate
0,2013.0,AK,Aleutians East,TSUN,72.972973,ERQK,69.933185,VLCN,61.797753,0,...,0.086842,0.225439,0.197076,15.935335,NaN,NaN,NaN,alaska,Aleutians East,1.4
31,2016.0,AK,Aleutians West,TSUN,95.945946,ERQK,89.850461,VLCN,68.539326,1,...,0.049121,0.287080,0.127676,16.805628,NaN,NaN,NaN,alaska,Aleutians West,1.8
62,2020.0,AK,Anchorage,ERQK,98.727331,AVLN,91.346154,WFIR,78.619154,2,...,0.047303,0.093670,0.090775,36.772161,NaN,NaN,NaN,alaska,Anchorage,3.3
93,2050.0,AK,Bethel,WFIR,78.396437,WNTW,75.119313,RFLD,74.673878,3,...,0.004500,0.010768,0.011090,12.728180,NaN,NaN,NaN,alaska,Bethel,9.4
124,2060.0,AK,Bristol Bay,WFIR,40.216354,ERQK,37.957366,VLCN,32.584270,4,...,0.007109,0.005924,0.053318,22.083981,NaN,NaN,NaN,alaska,Bristol Bay,8.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312814,56037.0,WY,Sweetwater,CWAV,73.560293,WFIR,62.360802,ERQK,57.842825,3262,...,0.008303,0.006695,0.164577,19.391717,Sweetwater,1411.0,6.2,wyoming,Sweetwater,4.3
312838,56039.0,WY,Teton,AVLN,100.000000,LNDS,92.547382,ERQK,91.059497,3263,...,0.002100,0.013244,0.141228,57.868509,Teton,4723.0,8.5,wyoming,Teton,2.3
312862,56041.0,WY,Uinta,WFIR,85.968820,ERQK,77.919185,WNTW,69.646834,3264,...,0.002445,0.005281,0.097115,17.955112,Uinta,1317.0,10.6,wyoming,Uinta,3.8
312886,56043.0,WY,Washakie,WFIR,56.761056,CWAV,46.516067,ERQK,43.175310,3265,...,0.001561,0.004294,0.139232,26.166329,Washakie,1317.0,0.9,wyoming,Washakie,5.0


In [38]:
final_df.columns

Index(['STCOFIPS', 'STATEABBRV', 'COUNTY', 'Risk_1_Type', 'Risk_1_Score',
       'Risk_2_Type', 'Risk_2_Score', 'Risk_3_Type', 'Risk_3_Score',
       'Unnamed: 0', 'state', 'county_risk', 'population', 'area',
       'risk_value', 'risk_score', 'risk_ratng', 'county_match',
       'county_price', 'price', 'county_inc', 'income', 'county_dem',
       'white_pct', 'black_pct', 'asian_pct', 'hispanic_pct', 'ed_pct',
       'county_tax', 'tax_paid_2022', 'tax_trend', 'state_name',
       'county_unemp', 'unemployment_rate'],
      dtype='object')

In [ ]:
columns_to_drop = [
    "STCOFIPS",
    "Unnamed: 0",
    "state",
    "county_risk",
    "county_tax",
    "county_match",
    "county_price",
    "county_inc",
    "state_name",
    "county_unemp",
    "county_dem",
    "risk_value",
]

# Drop the specified columns from the DataFrame
final_df_cleaned = final_df.drop(columns=columns_to_drop)

# Display the new DataFrame
final_df_cleaned

,STATEABBRV,COUNTY,Risk_1_Type,Risk_1_Score,Risk_2_Type,Risk_2_Score,Risk_3_Type,Risk_3_Score,population,area,...,price,income,white_pct,black_pct,asian_pct,hispanic_pct,ed_pct,tax_paid_2022,tax_trend,unemployment_rate
0,AK,Aleutians East,TSUN,72.972973,ERQK,69.933185,VLCN,61.797753,3374.0,15167.694230,...,NaN,65421.0,0.192398,0.086842,0.225439,0.197076,15.935335,NaN,NaN,1.4
31,AK,Aleutians West,TSUN,95.945946,ERQK,89.850461,VLCN,68.539326,5168.0,14258.993100,...,NaN,72234.0,0.302943,0.049121,0.287080,0.127676,16.805628,NaN,NaN,1.8
62,AK,Anchorage,ERQK,98.727331,AVLN,91.346154,WFIR,78.619154,290985.0,1966.338483,...,3.733779e+05,79183.0,0.543291,0.047303,0.093670,0.090775,36.772161,NaN,NaN,3.3
93,AK,Bethel,WFIR,78.396437,WNTW,75.119313,RFLD,74.673878,18633.0,46015.507070,...,NaN,48831.0,0.087217,0.004500,0.010768,0.011090,12.728180,NaN,NaN,9.4
124,AK,Bristol Bay,WFIR,40.216354,ERQK,37.957366,VLCN,32.584270,843.0,857.255664,...,NaN,169751.0,0.422986,0.007109,0.005924,0.053318,22.083981,NaN,NaN,8.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312814,WY,Sweetwater,CWAV,73.560293,WFIR,62.360802,ERQK,57.842825,42238.0,10597.074000,...,2.761582e+05,64115.0,0.765566,0.008303,0.006695,0.164577,19.391717,1411.0,6.2,4.3
312838,WY,Teton,AVLN,100.000000,LNDS,92.547382,ERQK,91.059497,23250.0,4259.833177,...,2.334408e+06,471751.0,0.803823,0.002100,0.013244,0.141228,57.868509,4723.0,8.5,2.3
312862,WY,Uinta,WFIR,85.968820,ERQK,77.919185,WNTW,69.646834,20412.0,2109.220457,...,2.883248e+05,49350.0,0.853007,0.002445,0.005281,0.097115,17.955112,1317.0,10.6,3.8
312886,WY,Washakie,WFIR,56.761056,CWAV,46.516067,ERQK,43.175310,7662.0,2265.341661,...,2.509363e+05,57474.0,0.815615,0.001561,0.004294,0.139232,26.166329,1317.0,0.9,5.0


In [ ]:
# Create a new variable for population density
final_df_cleaned["population_density"] = (
    final_df_cleaned["population"] / final_df_cleaned["area"]
)

# Drop the population and area columns from the DataFrame
final_df_cleaned = final_df_cleaned.drop(columns=["population", "area"])

# Display the new DataFrame
final_df_cleaned

,STATEABBRV,COUNTY,Risk_1_Type,Risk_1_Score,Risk_2_Type,Risk_2_Score,Risk_3_Type,Risk_3_Score,risk_score,risk_ratng,...,income,white_pct,black_pct,asian_pct,hispanic_pct,ed_pct,tax_paid_2022,tax_trend,unemployment_rate,population_density
0,AK,Aleutians East,TSUN,72.972973,ERQK,69.933185,VLCN,61.797753,4.231626,Very Low,...,65421.0,0.192398,0.086842,0.225439,0.197076,15.935335,NaN,NaN,1.4,0.222446
31,AK,Aleutians West,TSUN,95.945946,ERQK,89.850461,VLCN,68.539326,35.221126,Very Low,...,72234.0,0.302943,0.049121,0.287080,0.127676,16.805628,NaN,NaN,1.8,0.362438
62,AK,Anchorage,ERQK,98.727331,AVLN,91.346154,WFIR,78.619154,94.845689,Relatively Moderate,...,79183.0,0.543291,0.047303,0.093670,0.090775,36.772161,NaN,NaN,3.3,147.983169
93,AK,Bethel,WFIR,78.396437,WNTW,75.119313,RFLD,74.673878,16.926503,Very Low,...,48831.0,0.087217,0.004500,0.010768,0.011090,12.728180,NaN,NaN,9.4,0.404929
124,AK,Bristol Bay,WFIR,40.216354,ERQK,37.957366,VLCN,32.584270,0.159084,Very Low,...,169751.0,0.422986,0.007109,0.005924,0.053318,22.083981,NaN,NaN,8.2,0.983371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312814,WY,Sweetwater,CWAV,73.560293,WFIR,62.360802,ERQK,57.842825,6.617881,Very Low,...,64115.0,0.765566,0.008303,0.006695,0.164577,19.391717,1411.0,6.2,4.3,3.985817
312838,WY,Teton,AVLN,100.000000,LNDS,92.547382,ERQK,91.059497,68.501432,Relatively Low,...,471751.0,0.803823,0.002100,0.013244,0.141228,57.868509,4723.0,8.5,2.3,5.457960
312862,WY,Uinta,WFIR,85.968820,ERQK,77.919185,WNTW,69.646834,21.699014,Very Low,...,49350.0,0.853007,0.002445,0.005281,0.097115,17.955112,1317.0,10.6,3.8,9.677509
312886,WY,Washakie,WFIR,56.761056,CWAV,46.516067,ERQK,43.175310,2.513522,Very Low,...,57474.0,0.815615,0.001561,0.004294,0.139232,26.166329,1317.0,0.9,5.0,3.382271


In [41]:
final_df_cleaned.columns

Index(['STATEABBRV', 'COUNTY', 'Risk_1_Type', 'Risk_1_Score', 'Risk_2_Type',
       'Risk_2_Score', 'Risk_3_Type', 'Risk_3_Score', 'risk_score',
       'risk_ratng', 'price', 'income', 'white_pct', 'black_pct', 'asian_pct',
       'hispanic_pct', 'ed_pct', 'tax_paid_2022', 'tax_trend',
       'unemployment_rate', 'population_density'],
      dtype='object')

In [ ]:
# Rename columns
final_df_cleaned = final_df_cleaned.rename(
    columns={
        "STATEABBRV": "state",
        "COUNTY": "county",
        "unemployment_rate": "unemp_rate",
        "population_density": "pop_density",
    }
)

# Reorder columns to move 'price' to the first position
cols = list(final_df_cleaned.columns)
cols.insert(0, cols.pop(cols.index("price")))
final_df_cleaned = final_df_cleaned[cols]

# Reset the index
final_df_cleaned = final_df_cleaned.reset_index(drop=True)

# Display the new DataFrame
final_df_cleaned

,price,state,county,Risk_1_Type,Risk_1_Score,Risk_2_Type,Risk_2_Score,Risk_3_Type,Risk_3_Score,risk_score,...,income,white_pct,black_pct,asian_pct,hispanic_pct,ed_pct,tax_paid_2022,tax_trend,unemp_rate,pop_density
0,NaN,AK,Aleutians East,TSUN,72.972973,ERQK,69.933185,VLCN,61.797753,4.231626,...,65421.0,0.192398,0.086842,0.225439,0.197076,15.935335,NaN,NaN,1.4,0.222446
1,NaN,AK,Aleutians West,TSUN,95.945946,ERQK,89.850461,VLCN,68.539326,35.221126,...,72234.0,0.302943,0.049121,0.287080,0.127676,16.805628,NaN,NaN,1.8,0.362438
2,3.733779e+05,AK,Anchorage,ERQK,98.727331,AVLN,91.346154,WFIR,78.619154,94.845689,...,79183.0,0.543291,0.047303,0.093670,0.090775,36.772161,NaN,NaN,3.3,147.983169
3,NaN,AK,Bethel,WFIR,78.396437,WNTW,75.119313,RFLD,74.673878,16.926503,...,48831.0,0.087217,0.004500,0.010768,0.011090,12.728180,NaN,NaN,9.4,0.404929
4,NaN,AK,Bristol Bay,WFIR,40.216354,ERQK,37.957366,VLCN,32.584270,0.159084,...,169751.0,0.422986,0.007109,0.005924,0.053318,22.083981,NaN,NaN,8.2,0.983371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3265,2.761582e+05,WY,Sweetwater,CWAV,73.560293,WFIR,62.360802,ERQK,57.842825,6.617881,...,64115.0,0.765566,0.008303,0.006695,0.164577,19.391717,1411.0,6.2,4.3,3.985817
3266,2.334408e+06,WY,Teton,AVLN,100.000000,LNDS,92.547382,ERQK,91.059497,68.501432,...,471751.0,0.803823,0.002100,0.013244,0.141228,57.868509,4723.0,8.5,2.3,5.457960
3267,2.883248e+05,WY,Uinta,WFIR,85.968820,ERQK,77.919185,WNTW,69.646834,21.699014,...,49350.0,0.853007,0.002445,0.005281,0.097115,17.955112,1317.0,10.6,3.8,9.677509
3268,2.509363e+05,WY,Washakie,WFIR,56.761056,CWAV,46.516067,ERQK,43.175310,2.513522,...,57474.0,0.815615,0.001561,0.004294,0.139232,26.166329,1317.0,0.9,5.0,3.382271


In [ ]:
ppsf = pd.read_csv("data/ppsf.csv")

# Split the 'Region' column into 'county' and 'state' columns
ppsf[["county", "state"]] = ppsf["Region"].str.split(", ", expand=True)

# Remove the word 'County' from the 'county' column
ppsf["county"] = ppsf["county"].str.replace(" County", "")

# Drop the original 'Region' column
ppsf = ppsf.drop(columns=["Region"])

ppsf

,10/2024,county,state
0,173,Abbeville,SC
1,122,Acadia Parish,LA
2,184,Accomack,VA
3,281,Ada,ID
4,137,Adair,IA
...,...,...,...
2843,117,Young,TX
2844,266,Yuba,CA
2845,212,Yuma,AZ
2846,126,Yuma,CO


In [ ]:
df_temp = pd.merge(final_df_cleaned, ppsf, on=["county", "state"])
df_temp = df_temp.rename(columns={"10/2024": "price_sq"})

cols = list(df_temp.columns)
cols.insert(0, cols.pop(cols.index("price_sq")))
df_temp = df_temp[cols]

df_temp

,price_sq,price,state,county,Risk_1_Type,Risk_1_Score,Risk_2_Type,Risk_2_Score,Risk_3_Type,Risk_3_Score,...,income,white_pct,black_pct,asian_pct,hispanic_pct,ed_pct,tax_paid_2022,tax_trend,unemp_rate,pop_density
0,146,2.166933e+05,AL,Autauga,HWAV,87.082405,TRND,73.846643,HRCN,67.653656,...,53079.0,0.707117,0.193045,0.014846,0.036000,28.131469,531.0,8.1,1.9,96.260178
1,198,3.391105e+05,AL,Baldwin,LTNG,99.388674,HRCN,98.878421,WFIR,92.650334,...,60969.0,0.804666,0.077669,0.008754,0.054736,32.450286,796.0,10.9,2.0,112.985603
2,122,1.730130e+05,AL,Barbour,LTNG,82.303732,DRGT,80.687241,HRCN,77.164648,...,41531.0,0.439519,0.469809,0.004084,0.059866,11.153098,398.0,17.8,4.0,27.538003
3,129,2.013565e+05,AL,Bibb,TRND,71.969456,HWAV,67.228762,ERQK,61.692650,...,39835.0,0.737541,0.196923,0.001166,0.033194,11.913807,286.0,8.7,2.0,35.161208
4,137,2.203335e+05,AL,Blount,TRND,89.723194,LTNG,84.073359,HWAV,82.437162,...,45021.0,0.841546,0.013968,0.002942,0.097592,14.903610,464.0,11.5,1.8,89.763100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2821,144,2.761582e+05,WY,Sweetwater,CWAV,73.560293,WFIR,62.360802,ERQK,57.842825,...,64115.0,0.765566,0.008303,0.006695,0.164577,19.391717,1411.0,6.2,4.3,3.985817
2822,"1,229",2.334408e+06,WY,Teton,AVLN,100.000000,LNDS,92.547382,ERQK,91.059497,...,471751.0,0.803823,0.002100,0.013244,0.141228,57.868509,4723.0,8.5,2.3,5.457960
2823,146,2.883248e+05,WY,Uinta,WFIR,85.968820,ERQK,77.919185,WNTW,69.646834,...,49350.0,0.853007,0.002445,0.005281,0.097115,17.955112,1317.0,10.6,3.8,9.677509
2824,115,2.509363e+05,WY,Washakie,WFIR,56.761056,CWAV,46.516067,ERQK,43.175310,...,57474.0,0.815615,0.001561,0.004294,0.139232,26.166329,1317.0,0.9,5.0,3.382271
